## 策略简介CNN- 卷积神经网络

因子：样例因子  通过协方差和数据相关性统计---机器学习挖掘

因子是否标准化：否

标注：未来3日收益(不做离散化)

算法：CNN

类型：回归问题

训练集：2013-02-01  ~2019-10-30

测试集：2019-10-30~2021-12-20

选股依据：根据预测值降序排序买入

持股数：1

持仓天数：1
交易规则---尾盘14.50前涨停取消卖单------
 

In [6]:
# 本代码由可视化策略环境自动生成 2021年12月24日 16:41
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m4_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df =  input_1.read_pickle()
    feature_len = len(input_2.read_pickle())
    
    
    df['x'] = df['x'].reshape(df['x'].shape[0], int(feature_len), int(df['x'].shape[1]/feature_len))
    
    data_1 = DataSource.write_pickle(df)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m4_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m8_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df =  input_1.read_pickle()
    feature_len = len(input_2.read_pickle())
    
    
    df['x'] = df['x'].reshape(df['x'].shape[0], int(feature_len), int(df['x'].shape[1]/feature_len))
    
    data_1 = DataSource.write_pickle(df)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m8_post_run_bigquant_run(outputs):
    return outputs

# 用户的自定义层需要写到字典中，比如
# {
#   "MyLayer": MyLayer
# }
m5_custom_objects_bigquant_run = {
    
}

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m24_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    pred_label = input_1.read_pickle()
    df = input_2.read_df()
    df = pd.DataFrame({'pred_label':pred_label[:,0], 'instrument':df.instrument, 'date':df.date})
    df.sort_values(['date','pred_label'],inplace=True, ascending=[True,False])
    return Outputs(data_1=DataSource.write_df(df), data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m24_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m39_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 1
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = [1]
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 1
    context.options['hold_days'] = 1

# 回测引擎：每日数据处理函数，每天执行一次
def m39_handle_data_bigquant_run(context, data):
    # 获取当前持仓
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}
    
    today = data.current_dt.strftime('%Y-%m-%d')
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == today]
    try:
    #大盘风控模块，读取风控数据    
        benckmark_risk=ranker_prediction['bm_0'].values[0]
        if benckmark_risk > 0:
            for instrument in positions.keys():
                context.order_target(context.symbol(instrument), 0)
                print(today,'大盘风控止损触发,全仓卖出')
                return
    except:
        print('--！')
        
    #当risk为1时，市场有风险，全部平仓，不再执行其它操作    
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]
    #cash_for_buy = min(context.portfolio.portfolio_value/2,context.portfolio.cash)
    #cash_for_buy = context.portfolio.portfolio_value
    #print(ranker_prediction)
    #cash_for_buy = context.portfolio.portfolio_value
    cash_for_buy = context.portfolio.cash
    buy_instruments = list(ranker_prediction.instrument)
    sell_instruments = [instrument.symbol for instrument in context.portfolio.positions.keys()]
    to_buy = set(buy_instruments[:1]) - set(sell_instruments) 
    to_sell = set(sell_instruments) -  set(buy_instruments[:1])
   
    
    for instrument in to_sell:
        context.order_target(context.symbol(instrument), 0)
    for instrument in to_buy:
        context.order_value(context.symbol(instrument), cash_for_buy)

def m39_prepare_bigquant_run(context):


     # 获取st状态和涨跌停状态
    
    context.status_df = D.features(instruments =context.instruments,start_date = context.start_date, end_date = context.end_date, 
                           fields=['st_status_0','price_limit_status_0','price_limit_status_1'])

def m39_before_trading_start_bigquant_run(context, data):
    pass     
#     # 获取涨跌停状态数据
#     df_price_limit_status=context.status_df.set_index('date')
#     today=data.current_dt.strftime('%Y-%m-%d')
#     # 得到当前未完成订单
#     for orders in get_open_orders().values():
#         # 循环，撤销订单
#         for _order in orders:
#             ins=str(_order.sid.symbol)
#             try:
#                 #判断一下如果当日涨停，则取消卖单
#                 if  df_price_limit_status[df_price_limit_status.instrument==ins].price_limit_status_0.loc[today]>2 and _order.amount<0:
#                     cancel_order(_order)
#                     print(today,'尾盘涨停取消卖单',ins) 
#             except:
#                 continue
  
    
    

m1 = M.instruments.v2(
    start_date='2013-02-01',
    end_date='2019-10-30',
    market='CN_STOCK_A',
    instrument_list=' ',
    max_count=0
)

m21 = M.use_datasource.v1(
    instruments=m1.data,
    datasource_id='net_amount_CN_STOCK_A',
    start_date='',
    end_date=''
)

m22 = M.filter.v3(
    input_data=m21.data,
    expr='mf_net_amount_l>8000000',
    output_left_data=False
)

m23 = M.select_columns.v3(
    input_ds=m22.data,
    columns='date,instrument',
    reverse_select=False
)

m20 = M.use_datasource.v1(
    instruments=m1.data,
    datasource_id='bar1d_CN_STOCK_A',
    start_date='',
    end_date=''
)

m29 = M.join.v3(
    data1=m20.data,
    data2=m23.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m31 = M.auto_labeler_on_datasource.v1(
    input_data=m29.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
#shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
#clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
#all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
#where(shift(high, -1) == shift(low, -1), NaN, label)
# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(high, -3) / shift(open, -1)-1

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
#where(label>0.5, NaN, label)
#where(label<-0.5, NaN, label)
""",
    drop_na_label=True,
    cast_label_int=False,
    date_col='date',
    instrument_col='instrument'
)

m3 = M.input_features.v1(
    features="""return_5/return_20#43: 5天的收益率/20天的收益率
rank_amount_5#45：最近5日的成交额排名
avg_turn_10#46：平均10天的换手率
market_cap_float_0<280000000000#47：流通市值<280亿
pe_ttm_0>0#48：ttm pe市盈率要大于0
pb_lf_0#49:市净率
sum(mf_net_pct_main_0>0.12,30)>11#50：统计30天内主力流入占比大于12%的天数
fs_roa_ttm_0>5#51：总资产报酬率roa要大于5
fs_cash_ratio_0#52:现金流量
close_0>ts_max(close_0,56)#53：当日收盘价破 56天最高价（创新高）
ta_sma_10_0/ta_sma_30_0#56:   10天的sma线/30天的sma线
ta_sar_0# 58：SAR抛物线指标
swing_volatility_10_0/swing_volatility_60_0 #59:   10天的波动率/60天的波动率
ta_cci_14_0 #60：CCI -14天的指标
rank_return_3  #61:   3天收益率的 排名
mf_net_amount_0>mf_net_amount_1  #62:  判断 当日的资金流入净额>昨日资金流入净额
mf_net_amount_xl_0>mean(mf_net_amount_xl_0, 30)# 64：当天的超大单流入净量>平均30天内的超大单流入净量（30天超大单MA线）
cond4= (close_0-close_1)/close_1 >0.05#  65：当天涨幅>5%
(close_0-close_30)/close_30>1.25#  66:30天内的涨幅大于125%
(close_0-close_5)/close_5>1.16#  67：5天内的涨幅>116%
list_days_0>365#  68：上市天数>365天
ta_bbands_middleband_28_0 #69：布林带28天均线
cond28=sum(price_limit_status_0==3,80)>5  #70：统计80天内 涨停板的次数大于5"""
)

m25 = M.input_features.v1(
    features_ds=m3.data,
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
#st_status_0
#price_limit_status_0
#price_limit_status_1
#open_0
#close_0
#st_status_0
#fac2=where((open_0/high_0<0.97)&(high_0/close_0<1.04), 1,0)#(open_0/high_0<0.97)&(high_0/close_0<1.04)
#cond15=where(st_status_0==0,1,0)
#cond16=volume_0>volume_1
#cond17=ta_ma(close_0,5,derive='long')
#cond18=ta_trix(close_0, derive='long')
#fs_roe_ttm_0>5
#pe_ttm_0>0
#market_cap_float_0< 20000000000
#open_0
#close_0
#volume_2
#volume_0
#volume_1
#cond19=((volume_2/volume_1<2.5)|(high_0/close_0 <1.05))&(volume_2/volume_0>1)
#open_price/high_price<1) and (high_price/close_price<1.03)
#cond22=(open_0/high_0<0.97)&(high_0/close_0<1.04)
#close_0>open_0
#some321=ta_trix(close_0, derive='long')
#some321=ta_trix(close_1, derive='long')#新加的--可删除
#some321=ta_trix(close_2, derive='long')#新加的--可删除
#some321=ta_trix(close_3, derive='long')#新加的--可删除
#some321=ta_trix(close_4, derive='long')#新加的--可删除
#some456=ta_dma(close_0, 'long')#新加的，可删除
#some456=ta_dma(close_1, 'long')#新加的，可删除
#some456=ta_dma(close_2, 'long')#新加的，可删除
#cond30=mf_net_amount_main_0>0.1
open_1
close_1
close_0
high_1
open_0
low_0

price_limit_status_0
volume_0
open_0/close_1
cond3=low_0 > mean(close_0,20)
#(今日收盘价-昨日收盘价)/昨日收盘价*100%


cond1=ta_trix(close_0, derive='long')
cond2=ta_dma(close_0, 'long')
#----当日最低价 站稳60日线
cond3=low_0 > mean(close_0,20)
#(今日收盘价-昨日收盘价)/昨日收盘价*100%
cond4= (close_0-close_1)/close_1 >0.04
cond5=close_0>open_0
cond6=st_status_0==0
cond7=ta_macd(close_0,'long')
cond8=ta_ma(close_0,5, derive='long')"""
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m25.data,
    start_date='',
    end_date='',
    before_start_days=58
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m25.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m31.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m2 = M.filter.v3(
    input_data=m7.data,
    expr='cond4 and  cond6 and cond7 and cond8',
    output_left_data=False
)

m38 = M.features_short.v1(
    input_1=m3.data
)

m26 = M.dl_convert_to_bin.v2(
    input_data=m2.data,
    features=m38.data_1,
    window_size=12,
    feature_clip=2,
    flatten=True,
    window_along_col='instrument'
)

m4 = M.cached.v3(
    input_1=m26.data,
    input_2=m38.data_1,
    run=m4_run_bigquant_run,
    post_run=m4_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2019-10-30'),
    end_date=T.live_run_param('trading_date', '2021-12-20'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m25.data,
    start_date='',
    end_date='',
    before_start_days=58
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m25.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False
)

m13 = M.use_datasource.v1(
    instruments=m9.data,
    datasource_id='net_amount_CN_STOCK_A',
    start_date='',
    end_date=''
)

m14 = M.filter.v3(
    input_data=m13.data,
    expr='mf_net_amount_l>18000000',
    output_left_data=False
)

m35 = M.select_columns.v3(
    input_ds=m14.data,
    columns='date,instrument',
    reverse_select=False
)

m36 = M.join.v3(
    data1=m18.data,
    data2=m35.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m37 = M.filter.v3(
    input_data=m36.data,
    expr='cond4 and  cond6 and cond7 and cond8',
    output_left_data=False
)

m27 = M.dl_convert_to_bin.v2(
    input_data=m37.data,
    features=m38.data_1,
    window_size=12,
    feature_clip=2,
    flatten=True,
    window_along_col='instrument'
)

m8 = M.cached.v3(
    input_1=m27.data,
    input_2=m38.data_1,
    run=m8_run_bigquant_run,
    post_run=m8_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m6 = M.dl_layer_input.v1(
    shape='23,12',
    batch_shape='',
    dtype='float32',
    sparse=False,
    name=''
)

m10 = M.dl_layer_conv1d.v1(
    inputs=m6.data,
    filters=32,
    kernel_size='5',
    strides='1',
    padding='valid',
    dilation_rate=1,
    activation='relu',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m12 = M.dl_layer_maxpooling1d.v1(
    inputs=m10.data,
    pool_size=1,
    padding='valid',
    name=''
)

m32 = M.dl_layer_conv1d.v1(
    inputs=m12.data,
    filters=32,
    kernel_size='3',
    strides='1',
    padding='valid',
    dilation_rate=1,
    activation='relu',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m33 = M.dl_layer_maxpooling1d.v1(
    inputs=m32.data,
    pool_size=1,
    padding='valid',
    name=''
)

m28 = M.dl_layer_globalmaxpooling1d.v1(
    inputs=m33.data,
    name=''
)

m30 = M.dl_layer_dense.v1(
    inputs=m28.data,
    units=1,
    activation='linear',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m34 = M.dl_model_init.v1(
    inputs=m6.data,
    outputs=m30.data
)

m5 = M.dl_model_train.v1(
    input_model=m34.data,
    training_data=m4.data_1,
    optimizer='RMSprop',
    loss='mean_squared_error',
    metrics='mae',
    batch_size=10240,
    epochs=5,
    custom_objects=m5_custom_objects_bigquant_run,
    n_gpus=0,
    verbose='2:每个epoch输出一行记录'
)

m11 = M.dl_model_predict.v1(
    trained_model=m5.data,
    input_data=m8.data_1,
    batch_size=1024,
    n_gpus=0,
    verbose='2:每个epoch输出一行记录'
)

m24 = M.cached.v3(
    input_1=m11.data,
    input_2=m37.data,
    run=m24_run_bigquant_run,
    post_run=m24_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m44 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
#bm_0 = where(close/shift(close,5)-1<-0.05,1,0)

bm_0=where(ta_macd_dif(close,2,4,4)-ta_macd_dea(close,2,4,4)<0,1,0)"""
)

m43 = M.index_feature_extract.v3(
    input_1=m9.data,
    input_2=m44.data,
    before_days=100,
    index='000001.HIX'
)

m42 = M.select_columns.v3(
    input_ds=m43.data_1,
    columns='date,bm_0',
    reverse_select=False
)

m41 = M.join.v3(
    data1=m24.data_1,
    data2=m42.data,
    on='date',
    how='left',
    sort=False
)

m40 = M.sort.v4(
    input_ds=m41.data,
    sort_by='pred_label',
    group_by='date',
    keep_columns='--',
    ascending=False
)

m39 = M.trade.v4(
    instruments=m9.data,
    options_data=m40.sorted_data,
    start_date='',
    end_date='',
    initialize=m39_initialize_bigquant_run,
    handle_data=m39_handle_data_bigquant_run,
    prepare=m39_prepare_bigquant_run,
    before_trading_start=m39_before_trading_start_bigquant_run,
    volume_limit=0,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=100000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.SHA'
)


[2021-12-23 20:51:57.349150] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-12-23 20:51:57.358104] INFO: moduleinvoker: 命中缓存

[2021-12-23 20:51:57.359872] INFO: moduleinvoker: instruments.v2 运行完成[0.010731s].

[2021-12-23 20:51:57.371255] INFO: moduleinvoker: use_datasource.v1 开始运行..

[2021-12-23 20:51:57.392525] INFO: moduleinvoker: 命中缓存

...

[2021-12-23 20:52:24.729251] INFO: Performance: Simulated 522 trading days out of 522.

[2021-12-23 20:52:24.731227] INFO: Performance: first open: 2019-10-30 09:30:00+00:00

[2021-12-23 20:52:24.732747] INFO: Performance: last close: 2021-12-20 15:00:00+00:00

[2021-12-23 20:52:33.847809] INFO: moduleinvoker: backtest.v8 运行完成[32.611571s].

[2021-12-23 20:52:33.850707] INFO: moduleinvoker: trade.v4 运行完成[32.685729s].

In [ ]:
pd.DataFrame([DataSource(m5.data.id).read()]).to_pickle('/home/bigquant/work/userlib/model.csv')